# Getting started with `embeddings`

In [1]:
from langchain.embeddings import BedrockEmbeddings

## Embedding a sentence 

In [2]:
embeddings = BedrockEmbeddings()
text = "New Delhi is the capital of India"  

query_result = embeddings.embed_query(text)  

print(len(query_result))

1536


In [3]:
query_result

[0.87109375,
 0.30078125,
 -0.33984375,
 0.25195312,
 -0.875,
 0.16894531,
 0.64453125,
 -0.0005226135,
 -0.12597656,
 0.123046875,
 0.8515625,
 -0.53125,
 0.07080078,
 -0.609375,
 -0.28125,
 0.3359375,
 0.5390625,
 -0.16601562,
 0.37304688,
 -0.54296875,
 0.13574219,
 -0.515625,
 -0.6953125,
 -0.4296875,
 -0.61328125,
 -0.09765625,
 0.0038146973,
 -1.078125,
 0.13574219,
 -0.13183594,
 0.107910156,
 0.42382812,
 0.3203125,
 0.20410156,
 0.37890625,
 -0.765625,
 0.6484375,
 0.73828125,
 -0.35351562,
 -0.030273438,
 -0.45507812,
 0.12988281,
 0.3828125,
 0.69140625,
 -0.07763672,
 0.31640625,
 -0.37890625,
 0.016479492,
 0.19921875,
 0.6484375,
 -0.41015625,
 0.61328125,
 0.41601562,
 0.09814453,
 -0.41210938,
 0.21386719,
 -0.28515625,
 -0.03100586,
 -0.32617188,
 -0.22265625,
 -0.018676758,
 -0.3125,
 0.08154297,
 0.08886719,
 0.6953125,
 0.08154297,
 0.46484375,
 -0.56640625,
 -0.40039062,
 -0.35351562,
 0.5078125,
 0.82421875,
 0.6171875,
 1.203125,
 1.015625,
 0.08886719,
 -0.39257

## Embedding a group of sentences/document

In [4]:
embeddings = BedrockEmbeddings()
texts = ["New Delhi is the capital of India", 
         "Welcome to India", 
         "I am going to play football today"]

doc_vectors = embeddings.embed_documents(texts)

print(f"No. of vectors : {len(doc_vectors)}")
print(f"Dimension of each vectors : {[len(i) for i in doc_vectors]}")

No. of vectors : 3
Dimension of each vectors : [1536, 1536, 1536]


# Vector datastore (Amazon Aurora using `pgvector`)

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.1em;">
<b>Prerequisite:</b>
<ol>
    <li>Have an <b>Aurora cluster ready</b>.</li>
    <li>Create the <b>pgvector extension</b> on your Aurora PostgreSQL database (DB) cluster:
        <pre style="font-size: 1.1em;"><code>
        CREATE EXTENSION vector;
        </code></pre>
    </li>
</ol>
</div>


We can connect to the Aurora cluster and check 


```sql
-- SHOW the current database
SELECT current_database();

-- SHOW all the tables in the database
SELECT table_name
FROM postgres.information_schema.tables
WHERE table_schema = 'public';
```

## Importing few of the `libs`

In [5]:
import os
from dotenv import load_dotenv
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy

## Loading all the `env` variables 


In [6]:
load_dotenv()

True

## Create a `collection` and `connect` with the Vector store

In [7]:
# Collection Name 
COLLECTION_NAME = "my_collection"

# Connection String
CONNECTION_STRING = PGVector.connection_string_from_db_params(driver = os.getenv("PGVECTOR_DRIVER"),
                                                              user = os.getenv("PGVECTOR_USER"),                                      
                                                              password = os.getenv("PGVECTOR_PASSWORD"),                                  
                                                              host = os.getenv("PGVECTOR_HOST"),                                            
                                                              port = os.getenv("PGVECTOR_PORT"),                                          
                                                              database = os.getenv("PGVECTOR_DATABASE"),
                                                              )  

# Text Embedding model
embeddings = BedrockEmbeddings()

# Creating the VectorDB store instance   
my_vector_store = PGVector(collection_name=COLLECTION_NAME,
                           connection_string=CONNECTION_STRING,
                           embedding_function=embeddings,
                           distance_strategy = DistanceStrategy.EUCLIDEAN
                          )

At this point, `LangChain` will create **two** tables in the Aurora database:
- `langchain_pg_collection`
- `langchain_pg_embedding`


We can see these **two** tables 

![Vector Store on AWS](images/sql_lc_show_tables.png)

We can see our newly created `COLLECTION` **my_collection**

![Vector Store on AWS](images/sql_lc_show_tables2.png)

Let's see what we have inside the other table `langchain_pg_embedding` 

![Vector Store on AWS](images/sql_lc_show_tables3.png)

## Create some `vectors`

In [8]:
texts = ["New Delhi is the capital of India", 
         "Welcome to India", 
         "I am going to play football today"]

# Text --> Embeddings --> Vectors --> Aurora 
my_vector_store.from_texts(
                            texts=texts,
                            collection_name=COLLECTION_NAME,
                            connection_string=CONNECTION_STRING,
                            embedding=embeddings
                          );

In [9]:
len([0.87109375,0.30078125,-0.33984375,0.25195312,-0.875,0.16894531,0.64453125,-0.0005226135,-0.12597656,0.123046875,0.8515625,-0.53125,0.07080078,-0.609375,-0.28125,0.3359375,0.5390625,-0.16601562,0.37304688,-0.54296875,0.13574219,-0.515625,-0.6953125,-0.4296875,-0.61328125,-0.09765625,0.0038146973,-1.078125,0.13574219,-0.13183594,0.107910156,0.42382812,0.3203125,0.20410156,0.37890625,-0.765625,0.6484375,0.73828125,-0.35351562,-0.030273438,-0.45507812,0.12988281,0.3828125,0.69140625,-0.07763672,0.31640625,-0.37890625,0.016479492,0.19921875,0.6484375,-0.41015625,0.61328125,0.41601562,0.09814453,-0.41210938,0.21386719,-0.28515625,-0.03100586,-0.32617188,-0.22265625,-0.018676758,-0.3125,0.08154297,0.08886719,0.6953125,0.08154297,0.46484375,-0.56640625,-0.40039062,-0.35351562,0.5078125,0.82421875,0.6171875,1.203125,1.015625,0.08886719,-0.39257812,-0.51953125,0.41796875,-0.39257812,0.022216797,0.08691406,-0.7578125,0.546875,-0.040039062,-0.55078125,-0.37695312,0.29296875,0.0001296997,-0.004486084,0.25390625,0.68359375,0.29101562,-0.19433594,-0.26953125,-0.3203125,0.375,0.7421875,0.75,-0.04711914,-0.2421875,0.48828125,-0.28320312,0.44140625,0.2890625,0.76953125,0.18554688,0.38671875,-0.055419922,0.30078125,-0.12158203,-0.18945312,0.36523438,-0.27734375,-0.7734375,-0.03881836,0.47265625,0.122558594,-0.921875,-0.5390625,-0.421875,-0.024658203,-0.43359375,-0.24804688,0.6953125,0.234375,-0.33789062,-0.13867188,-0.16894531,-0.453125,-0.2734375,0.04711914,-0.4765625,0.118652344,-0.109375,0.33398438,-0.8359375,0.21484375,-0.734375,-0.61328125,0.39453125,-0.78125,0.58203125,-0.41796875,0.33984375,-0.22753906,-0.9765625,0.24609375,-0.5234375,-0.515625,-0.09814453,0.05517578,0.24707031,-0.3203125,-0.14453125,-0.026733398,0.02319336,0.88671875,-0.08105469,0.15039062,0.86328125,0.81640625,-0.49414062,-0.57421875,-0.76171875,-0.30859375,-0.24804688,0.5546875,0.36914062,-0.23828125,0.24707031,-0.5859375,-0.41015625,0.98828125,0.2109375,0.6015625,0.45703125,-0.51953125,0.12011719,-0.19628906,-0.083496094,0.6796875,0.18164062,0.44140625,0.328125,-0.65234375,-0.59765625,0.096191406,0.041503906,-0.1328125,0.17773438,0.71875,0.59375,-0.20507812,-0.66015625,0.20507812,-0.28710938,-0.62890625,-0.87109375,-0.21875,0.06225586,0.31445312,0.26953125,0.12060547,-0.38476562,0.045898438,0.05859375,-0.21777344,0.19433594,-0.20214844,-0.66796875,1.28125,-1.015625,-0.8984375,-0.54296875,-0.20800781,0.53515625,-0.3125,-0.30273438,-0.26757812,0.49804688,-0.8515625,-0.6796875,-1.453125,0.32226562,-0.27539062,0.55078125,-0.010681152,-0.00090789795,0.51171875,-0.15429688,1.734375,-0.43164062,0.52734375,-1.0234375,0.92578125,-0.46679688,0.64453125,0.032714844,0.4296875,-0.26757812,-0.8125,-0.16699219,0.48242188,-0.23730469,-0.63671875,0.3046875,0.13964844,-0.030883789,-0.4375,0.55078125,0.015075684,-0.122558594,0.30273438,-0.34765625,0.30273438,-0.00970459,0.11425781,0.51953125,0.057861328,0.21582031,-0.34570312,-0.16699219,-0.35351562,0.027954102,-0.875,0.40625,0.052978516,0.16308594,-0.546875,9.679794e-05,0.05126953,-0.22460938,0.27539062,0.36523438,0.22070312,-0.07324219,0.625,-0.25585938,-0.41796875,0.13476562,-0.45703125,0.17675781,-0.69140625,0.3671875,-1.7265625,0.11816406,1.2734375,0.13476562,0.16015625,-0.7265625,0.22265625,0.83984375,0.4296875,-0.07861328,0.20703125,0.38476562,0.31640625,-0.07324219,0.640625,-0.2578125,0.53125,0.08544922,-0.88671875,-0.079589844,-0.36328125,0.21582031,-0.38671875,0.4453125,0.55078125,-0.28320312,0.21972656,0.29492188,0.2265625,0.18261719,0.46289062,-0.1875,0.107910156,-0.5625,-0.6484375,0.056884766,-0.3515625,-0.86328125,-0.080078125,0.022827148,-0.23535156,0.06738281,-0.27734375,1.1328125,-0.5,-0.59765625,-1.265625,0.15527344,0.74609375,-0.36132812,-0.859375,0.59375,-0.17382812,-0.14550781,0.14257812,0.703125,0.08544922,0.24121094,-0.890625,-0.76953125,-0.71875,1.0078125,-1.078125,-0.13378906,1.0234375,0.26367188,0.12597656,-0.31640625,-0.640625,-0.038085938,-0.16796875,0.072753906,0.35546875,-0.44726562,-0.110839844,-0.61328125,-0.66015625,-0.16601562,-1.1015625,0.31640625,1.0234375,-0.7734375,-0.46875,0.19628906,0.6640625,-0.29492188,-0.25,-0.546875,0.07421875,-0.032470703,1.0234375,0.33203125,0.103515625,0.46679688,-0.28515625,0.1640625,-0.071777344,-0.061523438,-0.58203125,0.59765625,0.43554688,-0.58984375,0.45507812,0.6015625,-1.1796875,0.66796875,0.65625,-0.087890625,-0.25390625,-0.119140625,0.060058594,0.2734375,1.0703125,-0.4296875,0.3046875,0.022583008,0.33203125,0.70703125,0.97265625,0.50390625,0.4921875,-0.50390625,0.15136719,0.040039062,-0.11376953,-0.16601562,0.24121094,-0.57421875,-1.0859375,-0.14941406,-0.029907227,0.18457031,-0.8515625,-0.064453125,-0.17089844,0.099609375,0.1171875,0.12695312,-0.37109375,1.1484375,0.24902344,0.515625,-0.21777344,-0.13183594,0.53125,-0.42578125,0.24023438,-0.041992188,0.08203125,0.4921875,-0.66796875,0.15722656,0.51171875,0.12597656,-0.37890625,0.1015625,-0.25195312,-0.26367188,-0.5,0.1953125,-0.41992188,0.17382812,-0.07763672,0.9375,-0.5625,-0.1328125,-0.36914062,-0.64453125,-0.08105469,-0.46289062,-0.2578125,-0.83984375,0.01940918,0.4765625,0.013549805,0.21972656,0.032958984,-0.03540039,-0.4609375,-0.63671875,0.06738281,0.84765625,0.5625,0.24414062,-0.041259766,-0.26757812,-0.17382812,-0.32421875,0.92578125,-0.73828125,-0.12695312,-1.1171875,0.6875,-0.80859375,0.26757812,-0.27734375,0.82421875,1,-0.52734375,-0.07128906,-0.36132812,-0.100097656,-0.059814453,0.14355469,-0.546875,0.31640625,0.59765625,-0.35546875,-0.6640625,-0.94140625,0.15527344,-2.0742416e-05,-0.36132812,-0.025268555,-0.96484375,-0.0058898926,0.122558594,0.16308594,-0.65234375,-0.55078125,0.37109375,0.953125,-0.56640625,-0.07373047,-0.21484375,0.68359375,0.12451172,0.47460938,0.546875,0.7265625,0.375,-0.38476562,0.32226562,-0.29296875,-0.40039062,0.13867188,-0.40820312,-0.29492188,-0.91015625,-0.48828125,0.6171875,0.953125,0.37890625,-0.072753906,-0.43945312,-0.17773438,-0.18164062,-0.78125,0.31640625,0.8203125,0.49609375,-0.15234375,0.30078125,0.17578125,-0.62890625,1.453125,-1.1875,-1.234375,-0.0015182495,0.21972656,-0.3515625,-0.42773438,-1.234375,0.60546875,0.67578125,-0.51953125,0.12988281,-0.35742188,-0.24414062,-0.03515625,0.36914062,0.359375,1.046875,0.39648438,0.59765625,0.18652344,-0.011047363,0.24707031,-0.53125,0.48242188,0.24609375,0.08544922,-0.28125,-0.11816406,-0.18066406,0.07128906,-0.6328125,0.091308594,-0.4609375,-0.52734375,0.75,-0.41796875,0.12792969,0.9609375,-1.1640625,0.23730469,1.125,1.265625,0.9765625,-0.3203125,-0.25195312,-0.35546875,0.061523438,-0.65625,-1.0234375,-0.09423828,-0.609375,-0.3515625,0.33789062,-0.33203125,0.45117188,0.3125,-0.8828125,-1.21875,-0.9375,0.52734375,0.31835938,-0.45507812,-0.18457031,-0.38671875,0.12597656,0.31445312,-0.23925781,1.0546875,-0.54296875,0.15332031,-0.01586914,0.048095703,1.1171875,-0.86328125,-0.37695312,0.58203125,-0.65234375,-0.66015625,-0.16308594,0.06542969,-0.032714844,-0.23632812,-0.34765625,0.33203125,0.21582031,-0.37304688,-0.23046875,-0.6171875,0.578125,-0.017333984,0.56640625,-0.83984375,-0.024169922,0.023925781,-0.62109375,0.21484375,-0.91015625,0.63671875,-0.055908203,-0.5234375,-0.071777344,-0.78515625,0.075683594,-0.26757812,-0.44335938,0.33984375,0.671875,-0.59375,0.03881836,0.6484375,-0.24902344,-0.026367188,-0.43359375,0.010925293,-0.17480469,0.6484375,0.5,-0.07763672,0.359375,0.16992188,-0.010986328,0.61328125,0.14453125,0.8203125,-0.15917969,-0.44140625,0.33984375,0.43554688,-0.48828125,0.33789062,-0.9609375,-0.25976562,-0.20019531,-1.25,-0.25,-0.14648438,-0.076171875,-0.09082031,-0.23535156,0.032714844,-0.85546875,0.12158203,-0.15625,0.8046875,-0.58203125,0.30664062,-0.53515625,-0.14453125,0.16894531,0.084472656,0.33007812,0.24316406,1.09375,-0.375,1.2578125,-0.265625,0.1015625,0.25585938,0.78515625,-0.5859375,-0.29296875,-0.34765625,0.34765625,-0.328125,0.59765625,-0.2890625,0.51953125,-0.004119873,0.9765625,-0.47070312,0.66015625,0.07861328,-0.41992188,0.01965332,0.7265625,0.65234375,0.34179688,-0.15527344,0.51171875,-0.25195312,-0.20117188,-0.30859375,-0.67578125,0.2109375,-0.087402344,-0.28125,-0.043945312,0.0703125,-0.51953125,0.40820312,-0.06542969,-0.45898438,0.45898438,-0.46289062,-0.49609375,-0.31835938,-0.43164062,0.6015625,-0.13476562,0.890625,-0.390625,-0.2421875,-0.1015625,-0.2265625,0.060546875,0.4375,0.43945312,-0.40039062,0.16796875,-0.22265625,-0.4609375,0.14648438,0.20996094,-0.11279297,0.40039062,0.09277344,0.44140625,0.07714844,-0.19628906,-0.20214844,0.3515625,-0.005493164,0.453125,0.671875,0.1015625,0.068847656,-0.13476562,-0.009765625,0.99609375,1.0078125,0.07324219,-0.359375,-0.55078125,0.41992188,-0.115722656,1.09375,-0.50390625,0.028442383,-0.18652344,0.66796875,-0.14160156,-0.17382812,0.016967773,-0.66796875,-0.07373047,0.765625,0.96484375,-0.60546875,-0.21679688,-0.055664062,0.27929688,0.33398438,0.40429688,0.41796875,0.76953125,-0.25976562,-0.28515625,-0.21191406,-0.5703125,-0.15136719,-0.44140625,0.2265625,-0.22558594,0.16210938,0.46679688,-0.18652344,0.46484375,0.35351562,0.6171875,0.69140625,-0.3359375,0.43554688,-0.73828125,-0.27929688,-0.9296875,0.2421875,-0.734375,0.44726562,0.83203125,-0.24414062,0.5390625,-0.48828125,-0.0703125,-0.14257812,0.0012283325,0.6171875,0.6953125,-0.52734375,-0.54296875,0.36914062,0.58203125,0.056640625,-0.48046875,0.71875,0.2890625,0.033447266,-0.20800781,0.49804688,0.33007812,-0.19042969,-0.73828125,0.41210938,0.16308594,-0.18261719,0.66015625,0.06982422,0.5390625,-0.21289062,-0.02722168,-0.10986328,0.44921875,-0.10205078,-0.21386719,-0.05444336,-0.15820312,-0.33203125,-0.07861328,0.16796875,0.19628906,-0.21875,-0.11328125,-0.15625,0.4765625,0.23339844,1.0234375,-0.34960938,-0.24121094,-0.19238281,0.5859375,-0.68359375,-0.14257812,-0.091308594,0.55078125,1.359375,0.22949219,-0.049804688,0.3046875,-0.27539062,0.99609375,-0.828125,-0.8828125,0.28515625,-0.13476562,0.104003906,0.46289062,0.30273438,0.40820312,-0.0049743652,0.3671875,0.80859375,0.59375,0.07128906,0.3125,-0.055664062,0.76953125,0.34765625,-0.05078125,-0.296875,0.32421875,0.34570312,0.76953125,0.10986328,-0.71484375,-0.16015625,-0.49804688,-0.56640625,0.021850586,0.57421875,0.45703125,0.12207031,-0.1796875,0.55078125,-0.04736328,-0.6953125,1.171875,-0.58984375,0.24121094,-1.0859375,-0.37890625,0.16308594,0.06225586,0.74609375,-0.9375,0.044677734,0.4375,0.20410156,0.16015625,0.23144531,-0.73828125,-1.4921875,0.55859375,-0.70703125,0.13769531,-0.28515625,-0.0046081543,-0.42773438,-0.2734375,0.39453125,0.71484375,0.32617188,-0.0005340576,0.47460938,-0.02746582,-0.08203125,0.16699219,-0.56640625,0.0095825195,-0.30859375,-0.55859375,-0.5859375,0.20898438,0.38085938,0.00076675415,0.55078125,-0.72265625,0.20214844,-0.15625,0.024780273,0.6328125,-0.46875,0.5390625,-0.006011963,0.15136719,-1.140625,-0.22460938,-0.60546875,-0.15722656,0.0046691895,0.28320312,-0.34765625,-0.4765625,1.09375,-0.640625,-0.19335938,-0.34765625,1.2265625,0.49023438,0.29101562,-0.18359375,-1.34375,-0.087890625,-0.07421875,0.13183594,-0.28515625,0.35742188,-0.51953125,0.49414062,-0.21679688,-0.33984375,-0.04638672,-0.43359375,-0.21875,-0.053466797,0.18945312,0.41601562,1.046875,0.9765625,0.13671875,-0.875,0.109375,-0.014709473,-0.71484375,-0.36132812,-0.484375,0.13183594,-0.07470703,-0.14257812,1.09375,-1.40625,0.76171875,1.015625,0.41992188,-0.29882812,0.54296875,-0.3828125,-0.86328125,-0.15234375,-0.47851562,0.74609375,0.19335938,-0.8125,-1.6015625,-0.26953125,0.22167969,0.46484375,1.15625,0.12402344,0.15722656,0.45898438,0.35742188,-0.40234375,-0.2734375,0.62109375,0.58984375,0.13378906,0.46289062,-0.038330078,-0.32421875,0.671875,-0.10449219,0.04321289,-1.0703125,0.7734375,-0.57421875,-0.035888672,0.44335938,-0.16894531,0.0018310547,-0.027954102,-0.03466797,0.15722656,0.33789062,0.06225586,0.609375,-0.828125,-0.104003906,-0.8515625,0.30273438,-0.74609375,0.22851562,0.23535156,0.33203125,0.5234375,0.04345703,-0.14648438,-0.056884766,0.30664062,1.03125,0.24121094,-0.33203125,0.3125,0.18066406,1.1640625,-0.07519531,0.19042969,0.45117188,-0.11376953,-0.55859375,0.23242188,-0.04296875,-0.3828125,-0.31640625,0.18164062,0.5703125,-0.546875,1.296875,-0.24121094,-0.08935547,-0.09082031,-0.16015625,0.3125,0.14453125,0.20800781,0.41601562,-0.12451172,-0.46289062,-0.28125,-0.30664062,-0.088378906,0.09765625,1.5390625,-0.39648438,-0.78515625,0.40039062,0.33398438,-0.5703125,0.16699219,-0.24707031,0.41210938,-0.51171875,0.12402344,0.07763672,-0.08496094,0.515625,0.2890625,0.4765625,-0.23339844,-0.87109375,-0.67578125,0.62890625,-1.2265625,-0.859375,0.53125,-0.671875,0.003036499,-0.75,-0.8671875,-0.19628906,0.78125,-0.8203125,-0.38085938,-0.2265625,0.890625,0.34179688,-0.38671875,0.04272461,-0.25976562,-0.40039062,-0.15917969,0.38476562,0.4140625,1.0234375,-0.390625,0.94140625,0.5546875,0.10644531,0.37304688,-0.079589844,1.25,-0.921875,-0.49023438,0.49023438,-0.24511719,-0.60546875,0.171875,0.92578125,0.5078125,1.0703125,-0.5078125,0.49023438,0.36132812,-0.51171875,0.46679688,-0.034423828,0.16015625,0.78125,0.25390625,0.765625,0.45703125,-0.09863281,-0.48242188,-0.359375,-0.29492188,0.35546875,-0.86328125,0.13769531,0.17871094,-0.23632812,-0.16894531,-0.53515625,0.421875,0.15136719,-0.3671875,-0.171875,0.033691406,0.17871094,0.39648438,0.64453125,-0.30664062,0.375,-0.40820312,-0.328125,1.3125,0.40039062,-0.6015625,-0.15527344,0.28125,-0.20117188,0.26953125,0.22265625,0.33984375,0.05419922,-0.10888672,0.15820312,0.39453125,1.203125,0.118652344,0.578125,0.515625,0.3671875,0.076660156,0.25195312,-0.18554688,0.3515625,0.010498047,0.16015625,-1.0390625,-1.4375,-0.8828125,0.10888672,-0.546875,-0.9609375,-0.20605469,0.734375,-0.0033874512,-0.41015625,-0.5625,0.33007812,0.3046875,0.018676758,1.09375,-1.015625,-0.0012054443,0.036132812,-0.30859375,-0.38671875,-0.7890625,-0.57421875,0.017944336,-0.81640625,-0.5703125,0.71484375,0.8515625,0.0859375,0.008605957,-0.3203125,0.15332031,-0.48828125,0.72265625,-1.28125,0.08886719,0.29882812,-0.11279297,-0.59375,0.49023438,-0.34765625,-0.5390625,-0.265625,0.96875,0.39648438,-1.125,-0.17382812,0.25390625,0.044921875,-1.015625,0.5078125,0.42773438,0.27734375,0.50390625,-0.4609375,0.46679688,0.45898438,-0.020141602,-0.46679688,0.25976562,0.3125,-0.7578125,-0.49023438,-0.47460938,0.043945312,-0.30859375,0.037109375,0.47070312,-0.96875,0.296875,-0.54296875,0.8203125,-0.49414062,0.19628906,-0.05444336,-0.46289062,0.27734375,-0.19140625,0.36132812,2.125,-0.017578125,0.27929688,-0.25195312,0.06982422,-0.640625,-0.51953125,-0.76953125,0.20507812,-0.61328125,-1.28125,-0.02368164,-0.31835938,-0.62890625,-0.6484375,-0.60546875,0.29882812,0.43554688,0.76171875,-0.38085938,0.78125,-0.23046875,-0.296875,0.6875,0.4921875,-0.053955078,-0.15234375,0.060058594,-0.640625,-0.020141602,-0.6171875,-0.47851562,0.53515625,0.35351562,0.067871094,-0.82421875,0.18652344,0.90234375,-0.171875,-0.328125,-0.20605469,-0.65234375,0.03857422,0.62109375,-0.64453125,-0.84765625,0.171875,0.5234375,0.3359375,-0.036376953,0.36328125,0.22070312,0.7734375,0.0703125,0.28320312,-0.122558594,0.21875,-0.0625,-0.068847656,0.390625,0.12597656,-0.17480469,-0.66796875,-1.0546875,-0.60546875,0.65234375,0.030029297,0.15332031,1.25,0.75390625,-0.099121094,-0.21191406,-0.14941406,0.16503906,0.45898438,0.16992188,-0.091308594,0.67578125,-0.06347656,-0.390625,-0.6953125,-0.87109375,-0.515625,-0.48632812,0.3828125,0.46679688,0.4140625,-0.115234375,-1.09375,0.40039062,-0.18554688,0.115234375,-0.51953125,-0.6328125,-0.061523438,-0.022460938,-0.5078125,-0.28515625,-0.04638672,0.25976562,0.49609375,-0.9765625,-0.01953125,0.1015625,-0.14257812,0.3046875,0.53125,0.10449219,-0.21289062,-0.111328125,-0.13867188,0.76953125,0.54296875,0.030029297,-0.58203125,-0.15527344,0.14746094,0.61328125,0.43164062,-1.296875,-0.34375,0.23242188,0.51953125,0.5546875,-0.110839844,0.11328125,-0.515625,0.0066223145,-0.15722656,-0.51171875,0.21972656,0.87109375,0.4921875,-0.052001953,-0.51171875,0.63671875,-0.049072266,0.35351562,0.98828125,-0.83203125,0.78125,0.58203125,-0.34179688,0.03540039,0.36914062,-0.4375,-0.49804688,0.22558594,-0.1484375,0.14550781,0.29101562,-0.08105469,0.8828125,0.006164551,-0.0390625,-0.3671875,-1.09375,0.18554688,-0.14257812,-0.37109375,-0.13964844,-0.36914062,-0.014831543,0.0048828125,0.546875,0.89453125,0.44140625,0.6015625,-0.049316406,0.21582031,-1.078125,-0.00592041,-0.57421875,0.06640625,-0.11767578,-1.09375,-0.21289062,0.37695312,-0.33007812,0.3671875,0.58984375,0.05908203,-0.71484375,1.0625,0.37890625,0.15917969,-0.94921875,0.034423828,-0.5625,0.07910156,-0.28125,0.15429688,-0.78515625,0.39257812,0.84765625,-0.62109375,0.32421875,0.24804688,0.734375,-1.0078125,-0.41992188,0.5859375,-0.5859375,0.06347656,0.05883789,0.023803711,0.5859375,-0.22558594,-0.018432617,-0.22558594,0.7421875,0.036376953,0.46289062,0.0027160645,0.06689453,0.48632812,-0.29492188,0.33007812,-0.91015625,-0.41992188,-0.76953125,-0.13769531,0.37890625,0.68359375,-0.4609375,-0.80859375,0.609375,0.49023438,-0.061279297,-0.35546875,0.25195312,-0.84765625,0.5078125,-0.004180908,0.49414062,-0.029663086,-1.1484375,0.19726562,-0.24707031,-1.046875,0.052246094])

1536

Let's now check the `langchain_pg_embedding` table

![Vector Store on AWS](images/sql_lc_show_tables4.png)

#### Add few more vectors

In [10]:
texts = ['The sky is clear tonight.',
         'Cats are curious animals.',
         "It's raining in Paris.",
         'Learning Python can be fun.',
         'Coffee tastes better with friends.',
         'I live in Boston, its a beautiful city',
         'There is museum next to my home',
         'Music brings people together.',
         'The museum is closed on Mondays in few places',
         'In few places museums are open 7 days a week. like in my city']

# Text --> Embeddings --> Vectors --> Aurora 
my_vector_store.from_texts(
                            texts=texts,
                            collection_name=COLLECTION_NAME,
                            connection_string=CONNECTION_STRING,
                            embedding=embeddings
                          );


## Run `similarity search` with PGVector with distance.

In [11]:
my_vector_store.similarity_search(query = "Are museums open all the days in any city", k=4) 

[Document(page_content='In few places museums are open 7 days a week. like in my city'),
 Document(page_content='The museum is closed on Mondays in few places'),
 Document(page_content='There is museum next to my home'),
 Document(page_content='I live in Boston, its a beautiful city')]